5주차 예습과제에서는 6주차 내용인 GPT의 생성 능력을 맛보고자 합니다. 사용할 모델은 koalpaca라는 모델인데, 한국어 버전의 alpaca 모델입니다.

alpaca는 llama라는 모델을 instruction fine-tuning 해서 사람의 지시를 잘 따르는 모델입니다.

llama라는 모델은 LLaMA-13B의 경우 GPT-3보다 10배이상 작음에도 불구하고 대부분의 평가서 GPT-3보다 우수한 성능을 보이며, 더 나아가 LLaMA-65B의 경우 대부분의 벤치마크에서 Chinchilla, Gopher, GPT-3, PaLM와 유사하거나 더 뛰어난 결과를 보이는 훌륭한 모델입니다.

그러면 지금부터 koalpaca로 생성 모델을 한번 경험해보도록 하겠습니다!

In [1]:
# GPU 켜져 있는지 확인

!nvidia-smi

Mon Feb 12 09:36:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

peft는 parameter efficient fine-tuning의 약자로, 큰 모델을 코랩에서 돌릴 수 있도록 쪼개서 만든 것입니다.

오래 걸립니다!

In [4]:
from peft import PeftModel, PeftConfig

In [5]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

model-00001-of-00028.safetensors:   0%|          | 0.00/946M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

next token generation

In [6]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결하고 우리 삶을 혁신하는 '제 4차 산업혁명'의 시대가 도래하고 있습니다.​​​​​ ​​​�


 QA

In [7]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [8]:
gen('건강하게 살기 위한 세 가지 방법은?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:447: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 하나는 '감사 '인 것 같아요.또 하나는 '사랑'인 것 같아요.감사하는 마음은 긍정적인 사고를 만들어 줍니다.자신에게 주어지는 인생을 감사하고 사랑하는 마음으로살아가


top_k와 같은 parameter를 조절하면서 실험해보세요!

**Top-k**

예측한 다음 토큰들 중 확률이 가장 높은 k개 중 하나를 랜덤하게 선택.   
상대적으로 확률이 낮은 토큰도 선택될 수가 있음.   

 **Temperature**

예측한 다음 토큰들의 확률 분포를 변형시킴.   

*   0에 가까울수록 -> 확률 분포가 뾰족해짐 (확률이 큰 값은 더 크게, 작은 값은 더 작게), 정확도 높아짐

*   1 -> 확률 분포 그대로.

*   1보다 커질수록 -> 확률 분포가 평평해짐 (확률이 큰 값과 작은 값 사이의 차이가 줄어듦), 다양한 문장 생성   

Top-k Sampling / Top-p Sampling과 같이 사용해야 의미가 있음


https://beausty23.tistory.com/196


temperature=0.8, max_length=64, top_k=50

In [9]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제들을 풀고, 인류의 발전을 이끌어낼 것"이라고 말했다.<|endoftext|>


temperature=0.6, max_length=100, top_k=50

In [10]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.6, max_length=100, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결할 수 있을 것이라는 기대가 높아지고 있다. 인공지능은 인간의 지능을 모방하는 것에서 시작된 만큼, 지능의 근간이 되는 것이 무엇인지 살펴볼 필요가 있다. 인공지능은 인간의 지능을 모방하는 것에서 시작한다. 인공지능은 인간의 지능을 모방한 것이다. 인공지능의 핵심


temperature=1, max_length=100, top_k=80

In [11]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=1, max_length=100, top_k=80)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 복잡한 문제를 풀어낼 수 있을 것이라고 한다. 인공지능이 미래에 가져올 변화를 크게 3가지로 나누어 말한다. 1. 직업의 변화: 현재 인류가 하고 있는 직업의 대부분이 위협받게 된다. 단 예외는 있다. 회계나 변호사같은 아주 특수한 직업은 안전하게 유지할 수 있을 것으로 보고 있


temperature=1, max_length=64, top_k=50

In [13]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=1, max_length=64, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 거대한 난제들을 해결해 나갑니다. 인공지능과 함께하는 '더 나은 미래'의 청사진을 제시하는 책 [제3의 물결]은 미래사회에 대한 새로운 이정표를 제시


temperature=0.8, max_length=64, top_k=100

In [14]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=100)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 풀 수 있게 되었다. 인공지능(AI)은 현재 '의사'를 대신하는 역할을 하고 있고, 머지않아 '인간 건축가'도 대신하게 될 것이
